In [2]:
import pandas as pd
import numpy as np
import os
import requests 
import sys
import json
import datetime 

os.chdir(r'C:\Users\82102\OneDrive\바탕 화면\dataset\MFC')

import warnings
warnings.filterwarnings('ignore')

In [3]:
df_estate = pd.read_csv(r'공시지가_2022년.csv', encoding ='UTF8')
df_oil = pd.read_csv(r'서울시 주유소 현황_20220101.csv', encoding ='UTF8')

In [4]:
df_estate

,시도명,시군구명,법정동명,토지코드,공시지가(원/㎡),시군구코드,법정동코드,필지구분코드,필지구분명,본번,부번,기준년도,기준년월,주소
0,서울특별시,성북구,하월곡동,1130000000000000000.0,4837000,11290,13600,1,토지,90,1694,2022,2022-01-01,90-1694
1,서울특별시,성북구,하월곡동,1130000000000000000.0,5544000,11290,13600,1,토지,81,112,2022,2022-01-01,81-112
2,서울특별시,성북구,하월곡동,1130000000000000000.0,3605000,11290,13600,1,토지,81,214,2022,2022-01-01,81-214
3,서울특별시,성북구,하월곡동,1130000000000000000.0,3136000,11290,13600,1,토지,81,125,2022,2022-01-01,81-125
4,서울특별시,성북구,하월곡동,1130000000000000000.0,3245000,11290,13600,1,토지,90,1476,2022,2022-01-01,90-1476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903773,서울특별시,성북구,종암동,1130000000000000000.0,3765000,11290,13500,1,토지,66,16,2022,2022-01-01,66-16
903774,서울특별시,성북구,종암동,1130000000000000000.0,3485000,11290,13500,1,토지,28,266,2022,2022-01-01,28-266
903775,서울특별시,성북구,종암동,1130000000000000000.0,6850000,11290,13500,1,토지,29,12,2022,2022-01-01,29-12
903776,서울특별시,성북구,종암동,1130000000000000000.0,4125000,11290,13500,1,토지,79,317,2022,2022-01-01,79-317


In [5]:
df_estate['부번'] = df_estate['부번'].astype('str') # 부번 dtype이 int라 object로 형변환
df_estate['new_addrs'] = df_estate["본번"].map(str) + "-" + df_estate["부번"] # concat
df_estate

,시도명,시군구명,법정동명,토지코드,공시지가(원/㎡),시군구코드,법정동코드,필지구분코드,필지구분명,본번,부번,기준년도,기준년월,주소,new_addrs
0,서울특별시,성북구,하월곡동,1130000000000000000.0,4837000,11290,13600,1,토지,90,1694,2022,2022-01-01,90-1694,90-1694
1,서울특별시,성북구,하월곡동,1130000000000000000.0,5544000,11290,13600,1,토지,81,112,2022,2022-01-01,81-112,81-112
2,서울특별시,성북구,하월곡동,1130000000000000000.0,3605000,11290,13600,1,토지,81,214,2022,2022-01-01,81-214,81-214
3,서울특별시,성북구,하월곡동,1130000000000000000.0,3136000,11290,13600,1,토지,81,125,2022,2022-01-01,81-125,81-125
4,서울특별시,성북구,하월곡동,1130000000000000000.0,3245000,11290,13600,1,토지,90,1476,2022,2022-01-01,90-1476,90-1476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903773,서울특별시,성북구,종암동,1130000000000000000.0,3765000,11290,13500,1,토지,66,16,2022,2022-01-01,66-16,66-16
903774,서울특별시,성북구,종암동,1130000000000000000.0,3485000,11290,13500,1,토지,28,266,2022,2022-01-01,28-266,28-266
903775,서울특별시,성북구,종암동,1130000000000000000.0,6850000,11290,13500,1,토지,29,12,2022,2022-01-01,29-12,29-12
903776,서울특별시,성북구,종암동,1130000000000000000.0,4125000,11290,13500,1,토지,79,317,2022,2022-01-01,79-317,79-317


In [6]:
### oil data 주소 전처리 후 주유소 별 개별공시지가 세팅하기

In [7]:
addrs = df_oil['주소']

In [8]:
addrs.iloc[55] = '서울특별시 동대문구 서울시립대로 108'
addrs.iloc[138] = '서울특별시 마포구 대흥동 337-32'
addrs.iloc[233] = '서울특별시 영등포구 문래동3가 76-3'
addrs.iloc[240] = '서울특별시 영등포구 양평동5가 37'
addrs.iloc[264] = '서울 관악구 은천동 948-17'
addrs.iloc[298] = '서울특별시 금천구 독산동 292-7'
addrs.iloc[314] = '서울특별시 중랑구 망우동 147-23'
addrs.iloc[342] = '서울특별시 송파구 삼전동 9-23'
addrs.iloc[374] = '서울 서초구 방배동 810-22'
addrs.iloc[378] = '서울 서초구 내곡동 142-3' # row '378' of oil station is anomaly. kako api cannot return this address. we will drop this row later.
addrs.iloc[383] = '서울특별시 서초구 서초1동 1604-21'
addrs.iloc[54] = '서울 동대문구 답십리동 3-39'
addrs.iloc[467] = '서울 도봉구 창동 731-19'

In [9]:
# api받아서 locations에 저장
locations = []
for addr in addrs:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
    headers = {"Authorization": "KakaoAK 34261117a1efb76ce8fa2a300220db63"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

In [10]:
# 원하는 컬럼 리스트로 담고
# locations에 있는 값 할당
# try 실행 중 인덱싱, 컬럼에러 발생하면 중지 안하고 실행(예외처리)
real_address = []
city = [] 
town = []
bub_code = []
hangjeong_code = []
main_address_no = []
sub_address_no = []
longitude = []
latitude = []

for i in range(len(locations)):
    
    try:
        real_address.append(locations[i][0].get('address').get('address_name'))
        city.append(locations[i][0].get('address').get('region_2depth_name'))
        town.append(locations[i][0].get('address').get('region_3depth_name'))
        bub_code.append(locations[i][0].get('address').get('b_code'))
        hangjeong_code.append(locations[i][0].get('address').get('h_code'))
        main_address_no.append(locations[i][0].get('address').get('main_address_no'))
        sub_address_no.append(locations[i][0].get('address').get('sub_address_no'))
        longitude.append(locations[i][0].get('address').get('x'))
        latitude.append(locations[i][0].get('address').get('y'))

    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        real_address.append('없음')
        city.append('없음')
        town.append('없음')
        bub_code.append('없음')
        hangjeong_code.append('없음')
        main_address_no.append('없음')
        sub_address_no.append('없음')
        longitude.append('없음')
        latitude.append('없음')
        
    except AttributeError:
        real_address.append(locations[i][0].get('road_address').get('address_name'))
        city.append(locations[i][0].get('road_address').get('region_2depth_name'))
        town.append(locations[i][0].get('road_address').get('region_3depth_name'))
        bub_code.append(locations[i][0].get('road_address').get('b_code'))
        hangjeong_code.append(locations[i][0].get('road_address').get('h_code'))
        main_address_no.append(locations[i][0].get('road_address').get('main_address_no'))
        sub_address_no.append(locations[i][0].get('road_address').get('sub_address_no'))
        longitude.append(locations[i][0].get('road_address').get('x'))
        latitude.append(locations[i][0].get('road_address').get('y'))

378 번째 주소 못가져옴


Although we preprocessed 378th row, but could not get api solutions.

In [11]:
# 주유소데이터 도로명 -> 지번주소로,
# 개별공시지가 지번주소와 맞게
# 합쳐서 주유소 주소를 one key로 둘거임
tmp3 = np.array([real_address, city, town, bub_code, hangjeong_code, main_address_no, sub_address_no,
                latitude,longitude]).T
tmp3

array([['서울 용산구 후암동 448-103', '용산구', '후암동', ..., '103',
        '37.5544085670544', '126.977734582566'],
       ['서울 용산구 동자동 14-125', '용산구', '동자동', ..., '125',
        '37.5502005044121', '126.972417738531'],
       ['서울 용산구 갈월동 11-34', '용산구', '갈월동', ..., '34', '37.5470289447515',
        '126.972228457829'],
       ...,
       ['서울 도봉구 도봉동 601-55', '도봉구', '도봉동', ..., '55', '37.6744735831616',
        '127.044066665278'],
       ['서울 도봉구 도봉동 629', '도봉구', '도봉동', ..., '', '37.679014779402',
        '127.049750654086'],
       ['서울 도봉구 방학동 724-5', '도봉구', '방학동', ..., '5', '37.6622801187124',
        '127.047441496273']], dtype='<U21')

In [12]:
df_temp = pd.DataFrame(tmp3, columns = ['지번주소','구', '동', '법정코드','행정코드',
                                       '지번주번지', '지번부번지', '위도','경도'])
df_temp

,지번주소,구,동,법정코드,행정코드,지번주번지,지번부번지,위도,경도
0,서울 용산구 후암동 448-103,용산구,후암동,1117010100,1117051000,448,103,37.5544085670544,126.977734582566
1,서울 용산구 동자동 14-125,용산구,동자동,1117010700,1117053000,14,125,37.5502005044121,126.972417738531
2,서울 용산구 갈월동 11-34,용산구,갈월동,1117010400,1117053000,11,34,37.5470289447515,126.972228457829
3,서울 용산구 서계동 47-15,용산구,서계동,1117010800,1117055500,47,15,37.5523662854224,126.968993700509
4,서울 용산구 원효로2가 70-2,용산구,원효로2가,1117011300,1117056000,70,2,37.5355890312127,126.962708973419
...,...,...,...,...,...,...,...,...,...
468,서울 도봉구 도봉동 298-2,도봉구,도봉동,1132010800,1132052100,298,2,37.6883740291887,127.045327229383
469,서울 도봉구 도봉동 561-6,도봉구,도봉동,1132010800,1132052100,561,6,37.6843693792253,127.045522281038
470,서울 도봉구 도봉동 601-55,도봉구,도봉동,1132010800,1132052100,601,55,37.6744735831616,127.044066665278
471,서울 도봉구 도봉동 629,도봉구,도봉동,1132010800,1132052200,629,,37.679014779402,127.049750654086


In [13]:
df_temp.loc[df_temp['지번부번지'] == '', '지번부번지'] = 0
df_temp

,지번주소,구,동,법정코드,행정코드,지번주번지,지번부번지,위도,경도
0,서울 용산구 후암동 448-103,용산구,후암동,1117010100,1117051000,448,103,37.5544085670544,126.977734582566
1,서울 용산구 동자동 14-125,용산구,동자동,1117010700,1117053000,14,125,37.5502005044121,126.972417738531
2,서울 용산구 갈월동 11-34,용산구,갈월동,1117010400,1117053000,11,34,37.5470289447515,126.972228457829
3,서울 용산구 서계동 47-15,용산구,서계동,1117010800,1117055500,47,15,37.5523662854224,126.968993700509
4,서울 용산구 원효로2가 70-2,용산구,원효로2가,1117011300,1117056000,70,2,37.5355890312127,126.962708973419
...,...,...,...,...,...,...,...,...,...
468,서울 도봉구 도봉동 298-2,도봉구,도봉동,1132010800,1132052100,298,2,37.6883740291887,127.045327229383
469,서울 도봉구 도봉동 561-6,도봉구,도봉동,1132010800,1132052100,561,6,37.6843693792253,127.045522281038
470,서울 도봉구 도봉동 601-55,도봉구,도봉동,1132010800,1132052100,601,55,37.6744735831616,127.044066665278
471,서울 도봉구 도봉동 629,도봉구,도봉동,1132010800,1132052200,629,0,37.679014779402,127.049750654086


In [14]:
df_temp['지번주번지'] = df_temp['지번주번지'].astype('str')
df_temp['지번부번지'] = df_temp['지번부번지'].astype('str')
df_temp['address'] = df_temp[['지번주번지', '지번부번지']].apply('-'.join, axis=1)

df_temp

,지번주소,구,동,법정코드,행정코드,지번주번지,지번부번지,위도,경도,address
0,서울 용산구 후암동 448-103,용산구,후암동,1117010100,1117051000,448,103,37.5544085670544,126.977734582566,448-103
1,서울 용산구 동자동 14-125,용산구,동자동,1117010700,1117053000,14,125,37.5502005044121,126.972417738531,14-125
2,서울 용산구 갈월동 11-34,용산구,갈월동,1117010400,1117053000,11,34,37.5470289447515,126.972228457829,11-34
3,서울 용산구 서계동 47-15,용산구,서계동,1117010800,1117055500,47,15,37.5523662854224,126.968993700509,47-15
4,서울 용산구 원효로2가 70-2,용산구,원효로2가,1117011300,1117056000,70,2,37.5355890312127,126.962708973419,70-2
...,...,...,...,...,...,...,...,...,...,...
468,서울 도봉구 도봉동 298-2,도봉구,도봉동,1132010800,1132052100,298,2,37.6883740291887,127.045327229383,298-2
469,서울 도봉구 도봉동 561-6,도봉구,도봉동,1132010800,1132052100,561,6,37.6843693792253,127.045522281038,561-6
470,서울 도봉구 도봉동 601-55,도봉구,도봉동,1132010800,1132052100,601,55,37.6744735831616,127.044066665278,601-55
471,서울 도봉구 도봉동 629,도봉구,도봉동,1132010800,1132052200,629,0,37.679014779402,127.049750654086,629-0


In [15]:
df_oil.head()

,연번,자치구명,주유소명,주소
0,1,용산구,현대오일뱅크(주) 직영소월길주유소,서울특별시 용산구 소월로66
1,2,용산구,선익상사(주) 동자동주유소,서울특별시 용산구 한강대로 104길 6
2,3,용산구,현대오일뱅크㈜ 직영갈월동주유소,서울특별시 용산구 한강대로 322
3,4,용산구,서계주유소,서울특별시 용산구 청파로 367
4,5,용산구,㈜영원에너지 풍기주유소,서울특별시 용산구 원효로178


In [16]:
df_oil = pd.concat([df_oil, df_temp], axis =1)
df_oil

,연번,자치구명,주유소명,주소,지번주소,구,동,법정코드,행정코드,지번주번지,지번부번지,위도,경도,address
0,1,용산구,현대오일뱅크(주) 직영소월길주유소,서울특별시 용산구 소월로66,서울 용산구 후암동 448-103,용산구,후암동,1117010100,1117051000,448,103,37.5544085670544,126.977734582566,448-103
1,2,용산구,선익상사(주) 동자동주유소,서울특별시 용산구 한강대로 104길 6,서울 용산구 동자동 14-125,용산구,동자동,1117010700,1117053000,14,125,37.5502005044121,126.972417738531,14-125
2,3,용산구,현대오일뱅크㈜ 직영갈월동주유소,서울특별시 용산구 한강대로 322,서울 용산구 갈월동 11-34,용산구,갈월동,1117010400,1117053000,11,34,37.5470289447515,126.972228457829,11-34
3,4,용산구,서계주유소,서울특별시 용산구 청파로 367,서울 용산구 서계동 47-15,용산구,서계동,1117010800,1117055500,47,15,37.5523662854224,126.968993700509,47-15
4,5,용산구,㈜영원에너지 풍기주유소,서울특별시 용산구 원효로178,서울 용산구 원효로2가 70-2,용산구,원효로2가,1117011300,1117056000,70,2,37.5355890312127,126.962708973419,70-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,469,도봉구,현대오일뱅크㈜직영 도봉현대셀프주유소,서울특별시 도봉구 도봉로 941 (도봉동),서울 도봉구 도봉동 298-2,도봉구,도봉동,1132010800,1132052100,298,2,37.6883740291887,127.045327229383,298-2
469,470,도봉구,GS칼텍스㈜ 도봉주유소,서울특별시 도봉구 도봉로 895 (도봉동),서울 도봉구 도봉동 561-6,도봉구,도봉동,1132010800,1132052100,561,6,37.6843693792253,127.045522281038,561-6
470,471,도봉구,(주)송만에너지 도봉제일주유소,서울특별시 도봉구 도봉로 783 (도봉동),서울 도봉구 도봉동 601-55,도봉구,도봉동,1132010800,1132052100,601,55,37.6744735831616,127.044066665278,601-55
471,472,도봉구,노원교주유소,서울특별시 도봉구 마들로 776 (도봉동),서울 도봉구 도봉동 629,도봉구,도봉동,1132010800,1132052200,629,0,37.679014779402,127.049750654086,629-0


In [17]:
drop_list3 = ['연번', '자치구명', '주소', '행정코드', '지번주번지','지번부번지']
for i in drop_list3:
    df_oil = df_oil.drop(i, axis = 1)
df_oil

,주유소명,지번주소,구,동,법정코드,위도,경도,address
0,현대오일뱅크(주) 직영소월길주유소,서울 용산구 후암동 448-103,용산구,후암동,1117010100,37.5544085670544,126.977734582566,448-103
1,선익상사(주) 동자동주유소,서울 용산구 동자동 14-125,용산구,동자동,1117010700,37.5502005044121,126.972417738531,14-125
2,현대오일뱅크㈜ 직영갈월동주유소,서울 용산구 갈월동 11-34,용산구,갈월동,1117010400,37.5470289447515,126.972228457829,11-34
3,서계주유소,서울 용산구 서계동 47-15,용산구,서계동,1117010800,37.5523662854224,126.968993700509,47-15
4,㈜영원에너지 풍기주유소,서울 용산구 원효로2가 70-2,용산구,원효로2가,1117011300,37.5355890312127,126.962708973419,70-2
...,...,...,...,...,...,...,...,...
468,현대오일뱅크㈜직영 도봉현대셀프주유소,서울 도봉구 도봉동 298-2,도봉구,도봉동,1132010800,37.6883740291887,127.045327229383,298-2
469,GS칼텍스㈜ 도봉주유소,서울 도봉구 도봉동 561-6,도봉구,도봉동,1132010800,37.6843693792253,127.045522281038,561-6
470,(주)송만에너지 도봉제일주유소,서울 도봉구 도봉동 601-55,도봉구,도봉동,1132010800,37.6744735831616,127.044066665278,601-55
471,노원교주유소,서울 도봉구 도봉동 629,도봉구,도봉동,1132010800,37.679014779402,127.049750654086,629-0


In [18]:
df_table = pd.merge(left=df_oil, right=df_estate, how='left', left_on=['address','동'], right_on=['new_addrs','법정동명'])
df_table

,주유소명,지번주소,구,동,법정코드,위도,경도,address,시도명,시군구명,...,시군구코드,법정동코드,필지구분코드,필지구분명,본번,부번,기준년도,기준년월,주소,new_addrs
0,현대오일뱅크(주) 직영소월길주유소,서울 용산구 후암동 448-103,용산구,후암동,1117010100,37.5544085670544,126.977734582566,448-103,서울특별시,용산구,...,11170.0,10100.0,1.0,토지,448,103,2022.0,2022-01-01,448-103,448-103
1,선익상사(주) 동자동주유소,서울 용산구 동자동 14-125,용산구,동자동,1117010700,37.5502005044121,126.972417738531,14-125,서울특별시,용산구,...,11170.0,10700.0,1.0,토지,14,125,2022.0,2022-01-01,14-125,14-125
2,현대오일뱅크㈜ 직영갈월동주유소,서울 용산구 갈월동 11-34,용산구,갈월동,1117010400,37.5470289447515,126.972228457829,11-34,서울특별시,용산구,...,11170.0,10400.0,1.0,토지,11,34,2022.0,2022-01-01,11-34,11-34
3,서계주유소,서울 용산구 서계동 47-15,용산구,서계동,1117010800,37.5523662854224,126.968993700509,47-15,서울특별시,용산구,...,11170.0,10800.0,1.0,토지,47,15,2022.0,2022-01-01,47-15,47-15
4,㈜영원에너지 풍기주유소,서울 용산구 원효로2가 70-2,용산구,원효로2가,1117011300,37.5355890312127,126.962708973419,70-2,서울특별시,용산구,...,11170.0,11300.0,1.0,토지,70,2,2022.0,2022-01-01,70-2,70-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,현대오일뱅크㈜직영 도봉현대셀프주유소,서울 도봉구 도봉동 298-2,도봉구,도봉동,1132010800,37.6883740291887,127.045327229383,298-2,서울특별시,도봉구,...,11320.0,10800.0,1.0,토지,298,2,2022.0,2022-01-01,298-2,298-2
471,GS칼텍스㈜ 도봉주유소,서울 도봉구 도봉동 561-6,도봉구,도봉동,1132010800,37.6843693792253,127.045522281038,561-6,서울특별시,도봉구,...,11320.0,10800.0,1.0,토지,561,6,2022.0,2022-01-01,561-6,561-6
472,(주)송만에너지 도봉제일주유소,서울 도봉구 도봉동 601-55,도봉구,도봉동,1132010800,37.6744735831616,127.044066665278,601-55,서울특별시,도봉구,...,11320.0,10800.0,1.0,토지,601,55,2022.0,2022-01-01,601-55,601-55
473,노원교주유소,서울 도봉구 도봉동 629,도봉구,도봉동,1132010800,37.679014779402,127.049750654086,629-0,서울특별시,도봉구,...,11320.0,10800.0,1.0,토지,629,0,2022.0,2022-01-01,629-0,629-0


In [19]:
drop_list4 = ['지번주소', '구', '동', '법정코드', '시도명','토지코드','시군구코드','법정동코드', '필지구분코드', 
             '필지구분명', '본번', '부번', '기준년도', '기준년월', 'address', 'new_addrs']
for i in drop_list4:
    df_table = df_table.drop(i, axis = 1)
df_table

,주유소명,위도,경도,시군구명,법정동명,공시지가(원/㎡),주소
0,현대오일뱅크(주) 직영소월길주유소,37.5544085670544,126.977734582566,용산구,후암동,8280000.0,448-103
1,선익상사(주) 동자동주유소,37.5502005044121,126.972417738531,용산구,동자동,18850000.0,14-125
2,현대오일뱅크㈜ 직영갈월동주유소,37.5470289447515,126.972228457829,용산구,갈월동,15050000.0,11-34
3,서계주유소,37.5523662854224,126.968993700509,용산구,서계동,10330000.0,47-15
4,㈜영원에너지 풍기주유소,37.5355890312127,126.962708973419,용산구,원효로2가,13300000.0,70-2
...,...,...,...,...,...,...,...
470,현대오일뱅크㈜직영 도봉현대셀프주유소,37.6883740291887,127.045327229383,도봉구,도봉동,4205000.0,298-2
471,GS칼텍스㈜ 도봉주유소,37.6843693792253,127.045522281038,도봉구,도봉동,4395000.0,561-6
472,(주)송만에너지 도봉제일주유소,37.6744735831616,127.044066665278,도봉구,도봉동,5671000.0,601-55
473,노원교주유소,37.679014779402,127.049750654086,도봉구,도봉동,5380000.0,629-0


In [20]:
df_table = df_table[['주유소명', '위도', '경도', '시군구명', '법정동명', '공시지가(원/㎡)', '주소']]

In [21]:
df_table[df_table['법정동명'].isnull()]

,주유소명,위도,경도,시군구명,법정동명,공시지가(원/㎡),주소
178,㈜유턴상사유턴주유소,37.5572480682887,126.810028716951,NaN,NaN,NaN,NaN
380,지에스칼텍스(주)내곡주유소,없음,없음,NaN,NaN,NaN,NaN
452,㈜중앙에너비스 수서지점,37.4840308926021,127.094990086002,NaN,NaN,NaN,NaN


Due to the limitation of kakao api, the 3 rows could not be combined. => drop them

In [22]:
df_table = df_table.drop(df_table[df_table['법정동명'].isnull()].index, axis=0).reset_index(drop=True)

In [23]:
df_table = df_table.rename(columns = {'주유소명' : 'Gas_station', '위도' : 'Latitude', '경도' : 'Longitude', '시군구명' : 'District', '법정동명' : 'dong', '공시지가(원/㎡)' : 'estate', '주소' : 'address'})

In [24]:
df_table.columns = df_table.columns.str.upper()

In [25]:
df_table

,GAS_STATION,LATITUDE,LONGITUDE,DISTRICT,DONG,ESTATE,ADDRESS
0,현대오일뱅크(주) 직영소월길주유소,37.5544085670544,126.977734582566,용산구,후암동,8280000.0,448-103
1,선익상사(주) 동자동주유소,37.5502005044121,126.972417738531,용산구,동자동,18850000.0,14-125
2,현대오일뱅크㈜ 직영갈월동주유소,37.5470289447515,126.972228457829,용산구,갈월동,15050000.0,11-34
3,서계주유소,37.5523662854224,126.968993700509,용산구,서계동,10330000.0,47-15
4,㈜영원에너지 풍기주유소,37.5355890312127,126.962708973419,용산구,원효로2가,13300000.0,70-2
...,...,...,...,...,...,...,...
467,현대오일뱅크㈜직영 도봉현대셀프주유소,37.6883740291887,127.045327229383,도봉구,도봉동,4205000.0,298-2
468,GS칼텍스㈜ 도봉주유소,37.6843693792253,127.045522281038,도봉구,도봉동,4395000.0,561-6
469,(주)송만에너지 도봉제일주유소,37.6744735831616,127.044066665278,도봉구,도봉동,5671000.0,601-55
470,노원교주유소,37.679014779402,127.049750654086,도봉구,도봉동,5380000.0,629-0


In [32]:
df_table.to_csv(r'data_oil.csv', index=False)

In [33]:
df_oil = pd.read_csv(r'data_oil.csv', encoding ='UTF8')

In [34]:
df_gas = df_table.copy()

In [35]:
df_gas[['LATITUDE', 'LONGITUDE']] = df_oil[['LATITUDE', 'LONGITUDE']].astype(float)

In [36]:
df_DC = pd.read_csv(r'물류터미널정보_221001.csv',encoding ='UTF8')

In [37]:
df_gas

,GAS_STATION,LATITUDE,LONGITUDE,DISTRICT,DONG,ESTATE,ADDRESS
0,현대오일뱅크(주) 직영소월길주유소,37.554409,126.977735,용산구,후암동,8280000.0,448-103
1,선익상사(주) 동자동주유소,37.550201,126.972418,용산구,동자동,18850000.0,14-125
2,현대오일뱅크㈜ 직영갈월동주유소,37.547029,126.972228,용산구,갈월동,15050000.0,11-34
3,서계주유소,37.552366,126.968994,용산구,서계동,10330000.0,47-15
4,㈜영원에너지 풍기주유소,37.535589,126.962709,용산구,원효로2가,13300000.0,70-2
...,...,...,...,...,...,...,...
467,현대오일뱅크㈜직영 도봉현대셀프주유소,37.688374,127.045327,도봉구,도봉동,4205000.0,298-2
468,GS칼텍스㈜ 도봉주유소,37.684369,127.045522,도봉구,도봉동,4395000.0,561-6
469,(주)송만에너지 도봉제일주유소,37.674474,127.044067,도봉구,도봉동,5671000.0,601-55
470,노원교주유소,37.679015,127.049751,도봉구,도봉동,5380000.0,629-0


In [38]:
from haversine import haversine
from tqdm import notebook

distance_compare_list = [] # distance value 담을 리스트
for i in notebook.tqdm(range(df_DC.shape[0])):
    for j in range(df_gas.shape[0]):
        DC = (df_DC.Latitude[i], df_DC.Longitude[i]) 
        oil = (df_gas.LATITUDE[j], df_gas.LONGITUDE[j]) 
        result = haversine(DC, oil, unit='km')
        distance_compare_list.append(result)


# create distance df
distance = pd.DataFrame(data = distance_compare_list)
distance

  0%|          | 0/3 [00:00<?, ?it/s]

,0
0,8.970251
1,9.568763
2,9.713120
3,9.772973
4,11.006969
...,...
1411,25.294460
1412,24.849744
1413,23.746627
1414,24.267332


In [39]:
dist_eastDC = distance[:472].to_numpy() # 동부터미널까지 거리들
dist_westDC = distance[472:944].to_numpy() # 서부터미널까지 거리들
dist_koreaDC = distance[944:].to_numpy() # 한국터미널까지 거리들

In [40]:
df_gas['DIST_EASTDC'] = dist_eastDC
df_gas['DIST_WESTDC'] = dist_westDC
df_gas['DIST_KOREADC'] = dist_koreaDC

In [41]:
df_gas.head(3)

,GAS_STATION,LATITUDE,LONGITUDE,DISTRICT,DONG,ESTATE,ADDRESS,DIST_EASTDC,DIST_WESTDC,DIST_KOREADC
0,현대오일뱅크(주) 직영소월길주유소,37.554409,126.977735,용산구,후암동,8280000.0,448-103,8.970251,13.202286,11.678968
1,선익상사(주) 동자동주유소,37.550201,126.972418,용산구,동자동,18850000.0,14-125,9.568763,12.593357,11.494239
2,현대오일뱅크㈜ 직영갈월동주유소,37.547029,126.972228,용산구,갈월동,15050000.0,11-34,9.713120,12.454393,11.199948


#  QGIS 

In [43]:
df_Seoul = pd.read_excel(r'서울시 중심 격자.xlsx') # Seoul wkt_geom, latitude and longitude
df_pop = pd.read_table(r"2020년_가구_다사_100M.txt", sep = "^") # the number of population of each grid
school = pd.read_excel(r"학교_데이터.xlsx", sheet_name = '상세현황') # the number of school of each grid
apart = pd.read_excel(r"아파트_데이터.xlsx", sheet_name = '상세현황') # the number of apartments of each grid
market = pd.read_excel(r"상점_데이터.xlsx", sheet_name = '상세현황') # the number of stores of each grid
oil = pd.read_excel("서울시주유소_위경도.xlsx") #seoul gas station data

In [44]:
oil.rename(columns={'LATITUDE' : 'Latitude', 'LONGITUDE' : 'Longitude'}, inplace=True)

In [45]:
#change the format as df and extract columns only address, latitude and longitude
df_school = school.loc[:, ['주소', 'Latitude', 'Longitude']]
df_apart = apart.loc[:, ['주소', 'Latitude', 'Longitude']]
df_market = market.loc[:, ['주소', 'Latitude', 'Longitude']]
df_oil = oil.loc[:,  ['GAS_STATION', 'Latitude', 'Longitude']]

In [46]:
#Seoul multipolygon grid 100 * 100 (m)
df_Seoul.head()

,wkt_geom,GRID_100M_,경도,위도
0,MultiPolygon (((126.79139367692570772 37.54090...,다사374493,126.791955,37.541356
1,MultiPolygon (((126.78910424445719229 37.54359...,다사372496,126.789666,37.544046
2,MultiPolygon (((126.79138514660179737 37.54180...,다사374494,126.791947,37.542257
3,MultiPolygon (((126.79137661589790298 37.54270...,다사374495,126.791938,37.543159
4,MultiPolygon (((126.79023616444527534 37.54359...,다사373496,126.790798,37.544053


In [47]:
#sum of population of each grid
df_pop.head()

,연도,GRID_100M_,to_ga,인구
0,2020,다사000395,to_ga_001,0
1,2020,다사000398,to_ga_001,0
2,2020,다사000402,to_ga_001,5
3,2020,다사000408,to_ga_001,8
4,2020,다사000409,to_ga_001,23


In [48]:
#Merge dataset by grid
#If there is null value, fill it 0
df_LEFT_JOIN = pd.merge(df_Seoul, df_pop, left_on='GRID_100M_', right_on='GRID_100M_', how='left')
df_LEFT_JOIN.drop(columns = ['연도', 'to_ga'], inplace = True)
df_LEFT_JOIN = df_LEFT_JOIN.fillna(0)
df_LEFT_JOIN

,wkt_geom,GRID_100M_,경도,위도,인구
0,MultiPolygon (((126.79139367692570772 37.54090...,다사374493,126.791955,37.541356,0.0
1,MultiPolygon (((126.78910424445719229 37.54359...,다사372496,126.789666,37.544046,0.0
2,MultiPolygon (((126.79138514660179737 37.54180...,다사374494,126.791947,37.542257,0.0
3,MultiPolygon (((126.79137661589790298 37.54270...,다사374495,126.791938,37.543159,0.0
4,MultiPolygon (((126.79023616444527534 37.54359...,다사373496,126.790798,37.544053,0.0
...,...,...,...,...,...
61649,MultiPolygon (((127.08144454834838655 37.69461...,다사631662,127.082009,37.695066,0.0
61650,MultiPolygon (((127.08258387006598866 37.69371...,다사632661,127.083148,37.694169,0.0
61651,MultiPolygon (((127.082578817303542 37.6946172...,다사632662,127.083143,37.695070,0.0
61652,MultiPolygon (((127.08143948163113635 37.69551...,다사631663,127.082004,37.695967,0.0


In [49]:
#create sum of population by each grid (100m)
import haversine

df_LEFT_JOIN['points'] = df_LEFT_JOIN[['위도', '경도']].values.tolist()
df_oil['points'] = df_oil[['Latitude', 'Longitude']].values.tolist()
m = 100 #Set a radius of 100 meters
for i in range(len(df_oil)):
    print(i)
    oil_point = df_oil.loc[i, 'points']
    df_LEFT_JOIN['m'] = df_LEFT_JOIN['points'].apply(lambda x: haversine.haversine(oil_point, x, unit = 'm'))
    population = df_LEFT_JOIN[df_LEFT_JOIN['m'] <= m+1]['인구'].sum()
    df_oil.loc[i, 'population'] = population

df_oil

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,GAS_STATION,Latitude,Longitude,points,population
0,현대오일뱅크(주) 직영소월길주유소,37.554409,126.977735,"[37.5544085670544, 126.977734582566]",166.0
1,선익상사(주) 동자동주유소,37.550201,126.972418,"[37.5502005044121, 126.972417738531]",349.0
2,현대오일뱅크㈜ 직영갈월동주유소,37.547029,126.972228,"[37.5470289447515, 126.972228457829]",251.0
3,서계주유소,37.552366,126.968994,"[37.5523662854224, 126.968993700509]",102.0
4,㈜영원에너지 풍기주유소,37.535589,126.962709,"[37.5355890312127, 126.962708973419]",186.0
...,...,...,...,...,...
467,현대오일뱅크㈜직영 도봉현대셀프주유소,37.688374,127.045327,"[37.6883740291887, 127.045327229383]",16.0
468,GS칼텍스㈜ 도봉주유소,37.684369,127.045522,"[37.6843693792253, 127.045522281038]",180.0
469,(주)송만에너지 도봉제일주유소,37.674474,127.044067,"[37.6744735831616, 127.044066665278]",465.0
470,노원교주유소,37.679015,127.049751,"[37.679014779402, 127.049750654086]",55.0


In [50]:
#create sum of population by each grid (500m)
import haversine

df_LEFT_JOIN['points'] = df_LEFT_JOIN[['위도', '경도']].values.tolist()
df_oil['points'] = df_oil[['Latitude', 'Longitude']].values.tolist()
m = 500 #Set a radius of 500 meters
for i in range(len(df_oil)):
    print(i)
    oil_point = df_oil.loc[i, 'points']
    df_LEFT_JOIN['m'] = df_LEFT_JOIN['points'].apply(lambda x: haversine.haversine(oil_point, x, unit = 'm'))
    population = df_LEFT_JOIN[df_LEFT_JOIN['m'] <= m+1]['인구'].sum()
    df_oil.loc[i, 'population'] = population

df_oil

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,GAS_STATION,Latitude,Longitude,points,population
0,현대오일뱅크(주) 직영소월길주유소,37.554409,126.977735,"[37.5544085670544, 126.977734582566]",4465.0
1,선익상사(주) 동자동주유소,37.550201,126.972418,"[37.5502005044121, 126.972417738531]",7404.0
2,현대오일뱅크㈜ 직영갈월동주유소,37.547029,126.972228,"[37.5470289447515, 126.972228457829]",7069.0
3,서계주유소,37.552366,126.968994,"[37.5523662854224, 126.968993700509]",8374.0
4,㈜영원에너지 풍기주유소,37.535589,126.962709,"[37.5355890312127, 126.962708973419]",5401.0
...,...,...,...,...,...
467,현대오일뱅크㈜직영 도봉현대셀프주유소,37.688374,127.045327,"[37.6883740291887, 127.045327229383]",4651.0
468,GS칼텍스㈜ 도봉주유소,37.684369,127.045522,"[37.6843693792253, 127.045522281038]",9341.0
469,(주)송만에너지 도봉제일주유소,37.674474,127.044067,"[37.6744735831616, 127.044066665278]",7251.0
470,노원교주유소,37.679015,127.049751,"[37.679014779402, 127.049750654086]",9057.0


In [51]:
#create sum of population by each grid (1000m)
import haversine

df_LEFT_JOIN['points'] = df_LEFT_JOIN[['위도', '경도']].values.tolist()
df_oil['points'] = df_oil[['Latitude', 'Longitude']].values.tolist()
m = 1000 #Set a radius of 1000 meters
for i in range(len(df_oil)):
    print(i)
    oil_point = df_oil.loc[i, 'points']
    df_LEFT_JOIN['m'] = df_LEFT_JOIN['points'].apply(lambda x: haversine.haversine(oil_point, x, unit = 'm'))
    population = df_LEFT_JOIN[df_LEFT_JOIN['m'] <= m+1]['인구'].sum()
    df_oil.loc[i, 'population'] = population

df_oil

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

,GAS_STATION,Latitude,Longitude,points,population
0,현대오일뱅크(주) 직영소월길주유소,37.554409,126.977735,"[37.5544085670544, 126.977734582566]",15660.0
1,선익상사(주) 동자동주유소,37.550201,126.972418,"[37.5502005044121, 126.972417738531]",25467.0
2,현대오일뱅크㈜ 직영갈월동주유소,37.547029,126.972228,"[37.5470289447515, 126.972228457829]",26924.0
3,서계주유소,37.552366,126.968994,"[37.5523662854224, 126.968993700509]",30288.0
4,㈜영원에너지 풍기주유소,37.535589,126.962709,"[37.5355890312127, 126.962708973419]",27394.0
...,...,...,...,...,...
467,현대오일뱅크㈜직영 도봉현대셀프주유소,37.688374,127.045327,"[37.6883740291887, 127.045327229383]",12421.0
468,GS칼텍스㈜ 도봉주유소,37.684369,127.045522,"[37.6843693792253, 127.045522281038]",18592.0
469,(주)송만에너지 도봉제일주유소,37.674474,127.044067,"[37.6744735831616, 127.044066665278]",28101.0
470,노원교주유소,37.679015,127.049751,"[37.679014779402, 127.049750654086]",30149.0


In [52]:
##calculate the number of schools, apartment, market by grid (1000*1000)m
import haversine #globals : global variable -> for automation
m = 1000
df_oil['point'] = oil[['Latitude', 'Longitude']].values.tolist()
lst = ['school', 'apart', 'market']
for stat in lst:
    globals()[f'df_{stat}']['point'] = globals()[f'{stat}'][['Latitude', 'Longitude']].values.tolist()
    for i in range(len(oil)):
        oil_point = df_oil.loc[i, 'point']
        
        globals()[f'df_{stat}']['stat_per_m'] = globals()[f'df_{stat}']['point'].apply(lambda x: haversine.haversine(x, oil_point, unit = 'm'))
        df_oil.loc[i, stat] = len(globals()[f'df_{stat}'][globals()[f'df_{stat}']['stat_per_m'] <= m+1])

In [53]:
# number of schools each multipolygon
qgis_sch = df_oil.loc[:,['points','school']]
qgis_sch

,points,school
0,"[37.5544085670544, 126.977734582566]",5.0
1,"[37.5502005044121, 126.972417738531]",9.0
2,"[37.5470289447515, 126.972228457829]",9.0
3,"[37.5523662854224, 126.968993700509]",5.0
4,"[37.5355890312127, 126.962708973419]",9.0
...,...,...
467,"[37.6883740291887, 127.045327229383]",4.0
468,"[37.6843693792253, 127.045522281038]",4.0
469,"[37.6744735831616, 127.044066665278]",0.0
470,"[37.679014779402, 127.049750654086]",3.0


In [54]:
# number of apartments each multipolygon
qgis_apart = df_oil.loc[:,['points','apart']]
qgis_apart

,points,apart
0,"[37.5544085670544, 126.977734582566]",1.0
1,"[37.5502005044121, 126.972417738531]",2.0
2,"[37.5470289447515, 126.972228457829]",1.0
3,"[37.5523662854224, 126.968993700509]",2.0
4,"[37.5355890312127, 126.962708973419]",6.0
...,...,...
467,"[37.6883740291887, 127.045327229383]",1.0
468,"[37.6843693792253, 127.045522281038]",1.0
469,"[37.6744735831616, 127.044066665278]",0.0
470,"[37.679014779402, 127.049750654086]",1.0


In [55]:
# number of stores each multipolygon
qgis_store = df_oil.loc[:,['points','market']]
qgis_store

,points,market
0,"[37.5544085670544, 126.977734582566]",58.0
1,"[37.5502005044121, 126.972417738531]",6.0
2,"[37.5470289447515, 126.972228457829]",22.0
3,"[37.5523662854224, 126.968993700509]",4.0
4,"[37.5355890312127, 126.962708973419]",20.0
...,...,...
467,"[37.6883740291887, 127.045327229383]",0.0
468,"[37.6843693792253, 127.045522281038]",0.0
469,"[37.6744735831616, 127.044066665278]",4.0
470,"[37.679014779402, 127.049750654086]",2.0


In [56]:
# The area of each gas station was calculated using the QGIS program.
# To merge the area of the gas station with the independent variable data (DIST_EASTDC, DIST_WESTDC, DIST_KOREADC, POP1000, SCHOOL, APART, MARKET) obtained in the previous process, a column to be set as the merge criteria is required.
# However, since the name of the gas station is the same meaning but not the same, the QGIS program determined the latitude and longitude values of the lattice containing the gas station and merged them using a projection technique.
df_qgis = pd.read_csv('data_for_classification.csv')
df_qgis

,Unnamed: 0,GAS_STATION,DISTRICT,DONG,LATITUDE,LONGITUDE,AREA,ESTATE,DIST_EASTDC,DIST_WESTDC,DIST_KOREADC,POP100,POP500,POP1000,SCHOOL,APART,MARKET
0,0,현대오일뱅크(주) 직영소월길주유소,용산구,후암동,37.554409,126.977735,245.0,8280000.0,8.970251,13.202286,11.678968,166,4465,15660,5,1,58
1,1,선익상사(주) 동자동주유소,용산구,동자동,37.550201,126.972418,711.0,18850000.0,9.568763,12.593357,11.494239,349,7404,25467,9,2,6
2,2,현대오일뱅크㈜ 직영갈월동주유소,용산구,갈월동,37.547029,126.972228,700.0,15050000.0,9.713120,12.454393,11.199948,251,7069,26924,9,1,22
3,3,서계주유소,용산구,서계동,37.552366,126.968994,1010.0,10330000.0,9.772973,12.403577,11.855362,102,8374,30288,5,2,4
4,4,㈜영원에너지 풍기주유소,용산구,원효로2가,37.535589,126.962709,519.0,13300000.0,11.006969,11.272070,10.639303,186,5401,27394,9,6,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,464,현대오일뱅크㈜직영 도봉현대셀프주유소,도봉구,도봉동,37.688374,127.045327,1366.0,4205000.0,12.288585,26.954691,25.294460,16,4651,12421,4,1,0
465,465,GS칼텍스㈜ 도봉주유소,도봉구,도봉동,37.684369,127.045522,1640.7,4395000.0,11.849594,26.639204,24.849744,180,9341,18592,4,1,0
466,466,(주)송만에너지 도봉제일주유소,도봉구,도봉동,37.674474,127.044067,271.0,5671000.0,10.807711,25.756175,23.746627,465,7251,28101,0,0,4
467,467,노원교주유소,도봉구,도봉동,37.679015,127.049751,815.0,5380000.0,11.191130,26.466958,24.267332,55,9057,30149,3,1,2
